# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
cities_df = pd.read_csv("../cities.csv")

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

cities_df.head(3)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.8,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


## Generate Cities List

In [83]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

In [89]:
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# HOW SHOULD I GET CITY ID?
        #TRY #1
#         test = (cities_df.loc[cities_df['City'] == city, 'City_ID']).tolist()
#         city_id.append(test)

#     #TRY #2
#     for index, row in cities_df.iterrows():
#         city_id.append(row['City_ID'])
city_id = []
for city in cities:
#     city_id_df = cities_df.loc[cities_df['City'] == city, 'City_ID']
    city_id.append(cities_df.loc[cities_df['City'] == city, 'City_ID'])
city_id

[Series([], Name: City_ID, dtype: int64),
 Series([], Name: City_ID, dtype: int64),
 5    5
 Name: City_ID, dtype: int64,
 Series([], Name: City_ID, dtype: int64),
 13    13
 Name: City_ID, dtype: int64,
 75    75
 Name: City_ID, dtype: int64,
 392    392
 Name: City_ID, dtype: int64,
 21    21
 Name: City_ID, dtype: int64,
 15    15
 Name: City_ID, dtype: int64,
 Series([], Name: City_ID, dtype: int64),
 Series([], Name: City_ID, dtype: int64),
 366    366
 Name: City_ID, dtype: int64,
 Series([], Name: City_ID, dtype: int64),
 17    17
 Name: City_ID, dtype: int64,
 Series([], Name: City_ID, dtype: int64),
 Series([], Name: City_ID, dtype: int64),
 125    125
 Name: City_ID, dtype: int64,
 Series([], Name: City_ID, dtype: int64),
 Series([], Name: City_ID, dtype: int64),
 136    136
 Name: City_ID, dtype: int64,
 28    28
 Name: City_ID, dtype: int64,
 Series([], Name: City_ID, dtype: int64),
 Series([], Name: City_ID, dtype: int64),
 81    81
 Name: City_ID, dtype: int64,
 189    18

In [76]:
# Print the city count to confirm sufficient count
len(cities)
# print(len(city_id))


623

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).

In [90]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build query URL and request your results in Celsius
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [91]:
#TEST COMPLETE: 
cities = ['ostrovnoy', 'mahebourg', 'qaanaaq']

# List for holding temperature of each city
temp = []
humidity = []
cloudiness = []
windspeed = []
lat = []

# Loop through the list of cities and perform a request for data on each
for city in cities:
    
    # Get weather data
    weather_response = requests.get(query_url + city)
    weather_json = weather_response.json()
    
    # Get weather info from JSON response and add to the list
    temp.append(weather_json['main']['temp'])
    humidity.append(weather_json['main']['humidity'])
    cloudiness.append(weather_json['clouds']['all'])
    windspeed.append(weather_json['wind']['speed'])
    lat.append(weather_json['coord']['lat'])
    
    # print log of each city as it's being processed (city number and city name).
    print(f"city_numb - {city.title()} processed")

city_numb - Ostrovnoy processed
city_numb - Mahebourg processed
city_numb - Qaanaaq processed


[68.05, -20.41, 77.48]

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Are we creating a duplicate dataframe except we use API info and trying to see if it's the same as the cityPy csv file?
# I don't really understand. Do I put the print log into a new data frame and save it to a .csv?
cities_api_data_df = pd.DataFrame({
    "City Number": city_numb,
    "City": cities,
    "Latitude": lat,
    "Temperate": temp,
    "Humidity (%)": humidity,
    "Cloudiness (%)": cloudiness,
    "Wind Speed (mph)": windspeed
    })

# Save Data to csv
cities_data_df.to_csv("City_Data.csv")

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# humidity100_df = cities_api_data_df.loc[cities_api_data_df['Humidity (%)'] > 100, :]
# humidity100_df.head(3)

In [ ]:
#  Get the indices of cities that have humidity over 100%.

#https://stackoverflow.com/questions/41217310/get-index-of-a-row-of-a-pandas-dataframe-as-an-integer/42853445
humidity100_index = cities_api_data_df.loc[cities_api_data_df['Humidity (%)'] > 100, :].index.values.astype(int)[0]




In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression